<a href="https://colab.research.google.com/github/kumarashishraj/Movies-Recommender-Bot/blob/main/movies_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import pandas as pd
import numpy as np

In [133]:
movies = pd.read_csv ("tmdb_5000_movies.csv")


In [134]:
credits = pd.read_csv ("tmdb_5000_credits.csv")

In [135]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [136]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [137]:
movies = movies.merge(credits, on = 'title')

In [138]:
movies.shape

(4809, 23)

In [139]:
movies.head()


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [140]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [141]:
# we need to extract important features those will be required in our analysis and model building, here are a list of those columns
col = [ 'id', 'title','genres','keywords', 'overview', 'production_companies',  'cast', 'crew']
movies = movies[col]
movies.head()

,id,title,genres,keywords,overview,production_companies,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [142]:
# null values handling
movies.isnull().sum()

id                      0
title                   0
genres                  0
keywords                0
overview                3
production_companies    0
cast                    0
crew                    0
dtype: int64

In [143]:
# since only one record has a null value out of 5000 records we are dropping it
movies.dropna(inplace = True)


In [144]:
movies.isnull().sum()

id                      0
title                   0
genres                  0
keywords                0
overview                0
production_companies    0
cast                    0
crew                    0
dtype: int64

In [145]:
movies.head()

,id,title,genres,keywords,overview,production_companies,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [146]:
movies.iloc[-1].keywords

'[{"id": 1523, "name": "obsession"}, {"id": 2249, "name": "camcorder"}, {"id": 9986, "name": "crush"}, {"id": 11223, "name": "dream girl"}]'

In [147]:
import ast
ast.literal_eval

<function ast.literal_eval>

In [148]:
# since we only need the names of the keywords instead of id we clean the keywords column
# same goes with genre, production companies, cast, crew
def convert_keywords (obj):
  klist = []
  for key in ast.literal_eval(obj):
    klist.append(key['name'])
  return klist

def convert_productioncompanies (obj):
  company = []
  for i in ast.literal_eval(obj):
    company.append(i['name'])
  return company

def convert_cast (obj):
  cast = []
  cnt = 0
  for i in ast.literal_eval(obj):
    cast.append(i['name'])
    cnt = cnt + 1
    if cnt == 3:
      break
  return cast

def convert_crew (obj):
  crew = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      crew.append(i['name'])
      break
  return crew

def convert_genre (obj):
  genre = []
  for i in ast.literal_eval(obj):
    genre.append(i['name'])
  return genre



In [149]:
movies.keywords = movies.keywords.apply(convert_keywords)
movies.production_companies = movies.production_companies.apply(convert_productioncompanies)
movies.cast = movies.cast.apply (convert_cast)
movies.crew = movies.crew.apply (convert_crew)
movies.genres = movies.genres.apply (convert_genre)

In [150]:
movies.head()

,id,title,genres,keywords,overview,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,"[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,"[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [151]:
movies.overview = movies.overview.apply(str.split)

In [152]:
movies.head()

,id,title,genres,keywords,overview,production_companies,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...","[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send...","[Columbia Pictures, Danjaq, B24]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Following, the, death, of, District, Attorney...","[Legendary Pictures, Warner Bros., DC Entertai...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[John, Carter, is, a, war-weary,, former, mili...",[Walt Disney Pictures],"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [153]:
#we will now remove spaces between the words to treat them as a single entity
movies.cast = movies.cast.apply(lambda x : [i.replace(" ", "_") for i in x])
movies.crew = movies.crew.apply(lambda x : [i.replace(" ", "_") for i in x])
movies.production_companies = movies.production_companies.apply(lambda x : [i.replace(" ", "_") for i in x])
movies.keywords = movies.keywords.apply(lambda x : [i.replace(" ", "_") for i in x])


In [154]:
movies.genres = movies.keywords.apply(lambda x : [i.replace(" ", "_") for i in x])
movies.head()

,id,title,genres,keywords,overview,production_companies,cast,crew
0,19995,Avatar,"[culture_clash, future, space_war, space_colon...","[culture_clash, future, space_war, space_colon...","[In, the, 22nd, century,, a, paraplegic, Marin...","[Ingenious_Film_Partners, Twentieth_Century_Fo...","[Sam_Worthington, Zoe_Saldana, Sigourney_Weaver]",[James_Cameron]
1,285,Pirates of the Caribbean: At World's End,"[ocean, drug_abuse, exotic_island, east_india_...","[ocean, drug_abuse, exotic_island, east_india_...","[Captain, Barbossa,, long, believed, to, be, d...","[Walt_Disney_Pictures, Jerry_Bruckheimer_Films...","[Johnny_Depp, Orlando_Bloom, Keira_Knightley]",[Gore_Verbinski]
2,206647,Spectre,"[spy, based_on_novel, secret_agent, sequel, mi...","[spy, based_on_novel, secret_agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send...","[Columbia_Pictures, Danjaq, B24]","[Daniel_Craig, Christoph_Waltz, Léa_Seydoux]",[Sam_Mendes]
3,49026,The Dark Knight Rises,"[dc_comics, crime_fighter, terrorist, secret_i...","[dc_comics, crime_fighter, terrorist, secret_i...","[Following, the, death, of, District, Attorney...","[Legendary_Pictures, Warner_Bros., DC_Entertai...","[Christian_Bale, Michael_Caine, Gary_Oldman]",[Christopher_Nolan]
4,49529,John Carter,"[based_on_novel, mars, medallion, space_travel...","[based_on_novel, mars, medallion, space_travel...","[John, Carter, is, a, war-weary,, former, mili...",[Walt_Disney_Pictures],"[Taylor_Kitsch, Lynn_Collins, Samantha_Morton]",[Andrew_Stanton]


In [155]:
# now we need to concatenate the genre, keywords, overview, production comapnies, cast, crew in a same col called tags
movies['tags'] = movies.genres + movies.keywords + movies.overview + movies.production_companies + movies.cast + movies.crew 

In [156]:
new_col = [ 'id', 'title', 'tags']
movies = movies[new_col]
movies.tags = movies.tags.apply(" ".join)
movies.head()

,id,title,tags
0,19995,Avatar,culture_clash future space_war space_colony so...
1,285,Pirates of the Caribbean: At World's End,ocean drug_abuse exotic_island east_india_trad...
2,206647,Spectre,spy based_on_novel secret_agent sequel mi6 bri...
3,49026,The Dark Knight Rises,dc_comics crime_fighter terrorist secret_ident...
4,49529,John Carter,based_on_novel mars medallion space_travel pri...


In [157]:
# lower 
movies.tags = movies.tags.apply(str.lower)


In [158]:
movies.iloc[0].tags

'culture_clash future space_war space_colony society space_travel futuristic romance space alien tribe alien_planet cgi marine soldier battle love_affair anti_war power_relations mind_and_soul 3d culture_clash future space_war space_colony society space_travel futuristic romance space alien tribe alien_planet cgi marine soldier battle love_affair anti_war power_relations mind_and_soul 3d in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. ingenious_film_partners twentieth_century_fox_film_corporation dune_entertainment lightstorm_entertainment sam_worthington zoe_saldana sigourney_weaver james_cameron'

In [159]:
# steming is needed here to get the root word of any word
import nltk
stemmer = nltk.PorterStemmer()

In [160]:
def stem_sentence (text):
  l = []
  for i in text.split():
    l.append(stemmer.stem(i))

  return " ".join (l)

In [161]:
stemmer.stem('going')

'go'

In [162]:
movies.head()

,id,title,tags
0,19995,Avatar,culture_clash future space_war space_colony so...
1,285,Pirates of the Caribbean: At World's End,ocean drug_abuse exotic_island east_india_trad...
2,206647,Spectre,spy based_on_novel secret_agent sequel mi6 bri...
3,49026,The Dark Knight Rises,dc_comics crime_fighter terrorist secret_ident...
4,49529,John Carter,based_on_novel mars medallion space_travel pri...


In [163]:
movies.tags = movies.tags.apply(stem_sentence)

In [164]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=7000,stop_words='english')

In [165]:
cv.fit_transform(movies['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [166]:
vectors = cv.fit_transform(movies['tags']).toarray()

In [167]:
vectors.shape

(4806, 7000)

In [180]:
cv.get_feature_names()[: 50]

['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '1492_pictur',
 '15',
 '16',
 '17',
 '17th',
 '18',
 '18th',
 '18th_centuri',
 '19',
 '1910',
 '1920',
 '1930',
 '1940',
 '1944',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1971',
 '1974',
 '1976',
 '1980',
 '1985',
 '1990',
 '1995',
 '1999',
 '19th',
 '19th_centuri',
 '20',
 '200',
 '2003',
 '2009',
 '20th',
 '21_laps_entertain',
 '21st',
 '21st_centuri',
 '23',
 '24',
 '25',
 '2929_product']

In [169]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity (vectors)

In [170]:
similarity.shape

(4806, 4806)

In [171]:
similarity

array([[1.        , 0.        , 0.0280056 , ..., 0.02456254, 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.0135333 , 0.        ,
        0.01498727],
       [0.0280056 , 0.        , 1.        , ..., 0.01754116, 0.        ,
        0.        ],
       ...,
       [0.02456254, 0.0135333 , 0.01754116, ..., 1.        , 0.0354385 ,
        0.05111244],
       [0.        , 0.        , 0.        , ..., 0.0354385 , 1.        ,
        0.07849175],
       [0.        , 0.01498727, 0.        , ..., 0.05111244, 0.07849175,
        1.        ]])

In [182]:
list  (sorted(enumerate(similarity[0]), reverse = True, key = lambda x: x[1]))[:50]

[(0, 0.9999999999999999),
 (2409, 0.24005264491597397),
 (539, 0.22183742154872077),
 (1216, 0.21518834171247195),
 (1204, 0.20405350181565016),
 (507, 0.2000400120040014),
 (47, 0.19902714467551075),
 (3730, 0.1988529196671015),
 (778, 0.1930152894482308),
 (184, 0.1852396434087371),
 (74, 0.1847607809702561),
 (3538, 0.17943514064131835),
 (2333, 0.17383200648512995),
 (220, 0.1680336100833612),
 (61, 0.16736548175114463),
 (1089, 0.1669931442893732),
 (1920, 0.16451245200745981),
 (4, 0.16401525631526076),
 (838, 0.16374443516508944),
 (305, 0.15763738064909352),
 (1774, 0.15737789507292674),
 (843, 0.15495968363463283),
 (582, 0.15460560950731958),
 (3736, 0.15278308283803524),
 (4336, 0.15276156257835555),
 (1275, 0.14852213144650114),
 (4192, 0.14852213144650114),
 (373, 0.14686261420450203),
 (322, 0.14531393175878762),
 (579, 0.14408763192842225),
 (151, 0.14301401606039452),
 (3327, 0.14226859880591575),
 (1479, 0.14128765603607824),
 (2204, 0.13906560518810396),
 (3409, 0.137

In [173]:
def give_9_recommendations (movie):
  movie_index = movies[movies['title'] == movie].index[0]
  movie_list = list(sorted(enumerate(similarity[movie_index]), reverse = True, key = lambda x: x[1]))[1:9]
  for final in movie_list:
    print (movies.iloc[final[0]].title)


In [174]:
give_9_recommendations ('Iron Man')

Iron Man 2
Iron Man 3
Thor: The Dark World
The Avengers
Ant-Man
Captain America: Civil War
Avengers: Age of Ultron
Captain America: The First Avenger


In [175]:
# we will now need some variables and files to be dumped from the cloud server to the local pc for deployment 
import pickle

In [176]:
pickle.dump(movies.to_dict(), open('movie_dictionary.pkl', 'wb'))

In [177]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))